<a href="https://colab.research.google.com/github/kridtapon/WFO-Demand-Driven-KST-Strategy/blob/main/WFO_Demand_Driven_KST_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.6 MB/s eta 0:00:00


In [10]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

# Function to calculate the KST (Know Sure Thing) and Signal line
def calculate_kst(df, short_period=10, long_period=15, roc_short_period=5, roc_long_period=10):
    """
    Calculate KST (Know Sure Thing) and its Signal line.
    """
    # Rate of Change (ROC) for short and long periods
    roc_short = df['Close'].pct_change(periods=roc_short_period) * 100
    roc_long = df['Close'].pct_change(periods=roc_long_period) * 100
    roc_mid = df['Close'].pct_change(periods=short_period) * 100
    roc_longer = df['Close'].pct_change(periods=long_period) * 100

    # Smoothed ROC using weighted moving averages
    wma_short = roc_short.rolling(window=roc_short_period).mean()
    wma_long = roc_long.rolling(window=roc_long_period).mean()
    wma_mid = roc_mid.rolling(window=short_period).mean()
    wma_longer = roc_longer.rolling(window=long_period).mean()

    # KST = weighted sum of ROCs
    kst = (wma_short * 1) + (wma_long * 2) + (wma_mid * 3) + (wma_longer * 4)

    # Signal line: EMA of KST
    kst_signal = kst.ewm(span=9).mean()

    return kst, kst_signal

# Function to calculate the Demarker Indicator
def calculate_demarker(df, period=14):
    """
    Calculate the Demarker Indicator.
    """
    # Calculate the DeMarker Indicator
    dm_plus = np.maximum(df['High'].shift(1) - df['High'], 0)
    dm_minus = np.maximum(df['Low'] - df['Low'].shift(1), 0)

    # Calculate the smoothed values
    dem_plus = dm_plus.rolling(window=period).sum()
    dem_minus = dm_minus.rolling(window=period).sum()

    # Demarker Indicator
    dem = dem_plus / (dem_plus + dem_minus)

    return dem

# Define the stock symbol and time period
symbol = 'NRG' # VST AVGO
start_date = '2015-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Walk-forward optimization with KST and Demarker
def walk_forward_optimization_kst_demarker(df, start_year, end_year):
    results = []

    # Define dynamic ranges for parameters
    kst_short_period_range = range(5, 16, 2)  # Range for KST short periods
    kst_long_period_range = range(16, 31, 2)   # Range for KST long periods
    roc_short_period_range = range(5, 16, 2)  # Range for ROC short periods
    roc_long_period_range = range(16, 31, 2)   # Range for ROC long periods
    dem_period_range = range(5, 51, 2)        # Range for Demarker periods

    for test_year in range(start_year + 4, end_year + 1):
        train_start = test_year - 4
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_year]

        best_params = None
        best_performance = -np.inf

        # Loop through all combinations of KST, ROC, and Demarker parameters
        for params in itertools.product(kst_short_period_range, kst_long_period_range, roc_short_period_range, roc_long_period_range, dem_period_range):
            kst_short_period, kst_long_period, roc_short_period, roc_long_period, dem_period = params

            # Calculate KST and Demarker on the training data
            train_data['KST'], train_data['KST_Signal'] = calculate_kst(train_data, kst_short_period, kst_long_period, roc_short_period, roc_long_period)
            train_data['Demarker'] = calculate_demarker(train_data, dem_period)

            # Generate entry and exit signals
            entries = (train_data['KST'] > train_data['KST_Signal']) & (train_data['Demarker'] < 0.3)
            exits = (train_data['KST'] < train_data['KST_Signal']) & (train_data['Demarker'] > 0.7)

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = (kst_short_period, kst_long_period, roc_short_period, roc_long_period, dem_period)

        # Test with the best parameters on the test data
        test_data['KST'], test_data['KST_Signal'] = calculate_kst(test_data, best_params[0], best_params[1], best_params[2], best_params[3])
        test_data['Demarker'] = calculate_demarker(test_data, best_params[4])

        entries = (test_data['KST'] > test_data['KST_Signal']) & (test_data['Demarker'] < 0.3)
        exits = (test_data['KST'] < test_data['KST_Signal']) & (test_data['Demarker'] > 0.7)

        portfolio = vbt.Portfolio.from_signals(
            close=test_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params
        })

    return pd.DataFrame(results)


# Perform walk-forward optimization
results = walk_forward_optimization_kst_demarker(df, 2016, 2025)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)

# Combine signals into a single portfolio
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']
    yearly_data = df[df.index.year == year]

    # Apply KST and Demarker indicators
    yearly_data['KST'], yearly_data['KST_Signal'] = calculate_kst(yearly_data, params[0], params[1], params[2], params[3])
    yearly_data['Demarker'] = calculate_demarker(yearly_data, params[4])

    # Define entry/exit conditions
    entries = (yearly_data['KST'] > yearly_data['KST_Signal']) & (yearly_data['Demarker'] < 0.3)
    exits = (yearly_data['KST'] < yearly_data['KST_Signal']) & (yearly_data['Demarker'] > 0.7)

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for testing period only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest using the combined signals
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
<ipython-input-10-bb1bf88410f5>:87: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-bb1bf88410f5>:87: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-bb1bf88410f5>:88: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide


Walk-Forward Optimization Results:
   Year          Best_Params
0  2020   (5, 26, 5, 24, 13)
1  2021   (9, 26, 9, 24, 11)
2  2022   (9, 18, 9, 18, 11)
3  2023  (9, 28, 11, 20, 11)
4  2024    (5, 20, 5, 30, 7)
5  2025    (7, 30, 9, 16, 7)
Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           287083.413019
Total Return [%]                       187.083413
Benchmark Return [%]                   175.103187
Max Gross Exposure [%]                      100.0
Total Fees Paid                       5881.806095
Max Drawdown [%]                         23.68831
Max Drawdown Duration                       210.0
Total Trades                                   16
Total Closed Trades                            16
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%

<ipython-input-10-bb1bf88410f5>:148: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-bb1bf88410f5>:148: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-bb1bf88410f5>:149: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-b

In [11]:
# Filter Test Years
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Buy and Hold Performance Metrics
df_holding = df['Close']
pf = vbt.Portfolio.from_holding(df_holding, init_cash=100_000)
pf.stats()

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



,Close
Start,2020-01-02 00:00:00
End,2024-12-31 00:00:00
Period,1258
Start Value,100000.0
End Value,275103.18711
Total Return [%],175.103187
Benchmark Return [%],175.103187
Max Gross Exposure [%],100.0
Total Fees Paid,0.0
Max Drawdown [%],45.107312
